야후 API에서 제공하는 기업 설명을 따로 supabase의 저장소에 추가

In [14]:
#라이브러리
##기본 작업
import os
import pandas as pd
import numpy as np
import time
import json
from datetime import datetime, timedelta
##뉴스스크랩
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from dateutil.parser import parse
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
## API 구동
import http.client
import uuid
from supabase import create_client, Client
from google import genai
from google.genai import types

# supabase 연결

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client
# 환경 변수 사용
# 상위 폴더의 .env 파일 경로를 지정하여 로드
# 그러나 보안을 위해 업로드 하지 않음
load_dotenv(dotenv_path='../.env')
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

# 야후 API

In [7]:
import yfinance as yf

# 한국 주식 목록
Name_KR=['Samsung', 'SK Hynix', 'Samsung Biologics', 'LG Energy Solution', 'Hyundai', 'KB Financial Group', 'Celltrion', 'Doosan Enerbility', 'Kia', 'Hanwha Aerospace', 'Naver', 'HD Hyundai Heavy Industries', 'Shinhan Financial Group', 'Hyundai Mobis', 'Kakao', 'HMM', 'Hana Financial Group', 'POSCO', 'Korea Electric Power', 'Samsung Life Insurance', 'Hanwha Ocean (DSME)', 'HD Korea Shipbuilding & Offshore Engineering', 'SK Square', 'Meritz Financial Group', 'Alteogen', 'Hyundai Rotem Company', 'LG Chem', 'SK Innovation', 'Woori Financial Group', 'Krafton', 'HD Hyundai Electric', 'Industrial Bank of Korea (IBK)', 'Korea Zinc', 'Samsung SDI', 'KakaoBank', 'KT Corporation', 'KT&G (Korea Tobacco)', 'Samsung Heavy Industries', 'LG Electronics', 'LG Corp', 'SK Telecom', 'HYBE', 'SK Group', 'LIG Nex1', 'POSCO Chemical', 'Kakao Pay', 'Samsung Electro-Mechanics', 'Hyundai Glovis', 'Ecopro BM', 'Hanwha Systems', 'Samyang Foods', 'HANMI Semiconductor', 'Doosan', 'Korean Air Lines', 'Posco International', 'HD Hyundai Marine Solution', 'Korea Aerospace Industries', 'Amorepacific', 'Yuhan', 'S-OIL', 'Coway', 'Ecopro', 'Hanwha Solutions', 'Doosan Bobcat', 'Hankook Tire', 'Netmarble', 'LG Household & Health Care', 'LG Display', 'LS Corp.', 'ORION', 'Korea Gas', 'CJ Group', 'NCsoft', 'Kangwon Land', 'LEENO Industrial', 'F&F Co', 'SM Entertainment', 'Kumho Petrochemical', 'Hanmi Pharmaceutical', 'LOTTE Chemical', 'SHIFT UP', 'Pearl Abyss', 'JYP Entertainment', 'KCC Corp', 'Nongshim', 'Fila', 'BGF Retail', 'Pan Ocean', 'LOTTE Corporation', 'Asiana Airlines', 'Hansol Chemical', 'Shinsegae', 'YG Entertainment', 'Doosan Fuel Cell', 'Kakao Games', 'HiteJinro', 'Kumho Tire', 'Ottogi', 'SD BioSensor', 'DoubleUGames']
Ticker_KR=['005930.KS', '000660.KS', '207940.KS', '373220.KS', 'HYMTF', 'KB', '068270.KS', '034020.KS', '000270.KS', '012450.KS', '035420.KS', '329180.KS', 'SHG', '012330.KS', '035720.KS', '011200.KS', '086790.KS', 'PKX', 'KEP', '032830.KS', '042660.KS', '009540.KS', '402340.KS', '138040.KS', '196170.KQ', '064350.KS', '051910.KS', '096775.KS', 'WF', '259960.KS', '267260.KS', '024110.KS', '010130.KS', '006405.KS', '323410.KS', 'KT', '033780.KS', '010140.KS', '066570.KS', '003550.KS', 'SKM', '352820.KS', '034730.KS', '079550.KS', '003670.KS', '377300.KS', '009155.KS', '086280.KS', '247540.KQ', '272210.KS', '003230.KS', '042700.KS', '000150.KS', '003495.KS', '047050.KS', '443060.KS', '047810.KS', '090430.KS', '000100.KS', '010950.KS', '021240.KS', '086520.KQ', '009830.KS', '241560.KS', '161390.KS', '251270.KS', '051905.KS', 'LPL', '006260.KS', '271560.KS', '036460.KS', '001040.KS', '036570.KS', '035250.KS', '058470.KQ', '383220.KS', '041510.KQ', '011780.KS', '008930.KS', '011170.KS', '462870.KS', '263750.KQ', '035900.KQ', '002380.KS', '004370.KS', '081660.KS', '282330.KS', '028670.KS', '004990.KS', '020560.KS', '014680.KS', '004170.KS', '122870.KQ', '336260.KS', '293490.KQ', '000080.KS', '073240.KS', '007310.KS', '137310.KS', '192080.KS']
# 미국 주식 목록
Name_US=['NVIDIA', 'Microsoft', 'Apple', 'Amazon', 'Alphabet (Google)', 'Meta Platforms (Facebook)', 'Broadcom', 'Berkshire Hathaway', 'Tesla', 'JPMorgan Chase', 'Walmart', 'Visa', 'Eli Lilly', 'Oracle', 'Netflix', 'Mastercard', 'Exxon Mobil', 'Costco', 'Procter & Gamble', 'Johnson & Johnson', 'Home Depot', 'Bank of America', 'AbbVie', 'Palantir', 'Coca-Cola', 'UnitedHealth', 'Philip Morris International', 'Cisco', 'T-Mobile US', 'Wells Fargo', 'IBM', 'General Electric', 'Salesforce', 'Chevron', 'Abbott Laboratories', 'Morgan Stanley', 'American Express', 'AMD', 'Walt Disney', 'Goldman Sachs', 'Intuit', 'ServiceNow', 'McDonald', 'AT&T', 'Merck', 'Texas Instruments', 'Uber', 'Intuitive Surgical', 'RTX', 'Blackstone Group', 'Caterpillar', 'Booking Holdings (Booking.com)', 'Pepsico', 'Verizon', 'QUALCOMM', 'BlackRock', 'Charles Schwab', 'Citigroup', 'Boeing', 'S&P Global', 'Thermo Fisher Scientific', 'Adobe', 'Amgen', 'Honeywell', 'Boston Scientific', 'Progressive', 'Applied Materials', 'Nextera Energy', 'Stryker Corporation', 'Danaher', 'Pfizer', 'Capital One', 'Union Pacific Corporation', 'GE Vernova', 'Deere & Company (John Deere)', 'TJX Companies', 'Gilead Sciences', 'Micron Technology', 'Palo Alto Networks', 'Comcast', 'Arista Networks', 'KKR & Co.', 'CrowdStrike', "Lowe's Companies", 'Lam Research', 'Automatic Data Processing', 'KLA', 'Analog Devices', 'Amphenol', 'ConocoPhillips', 'Vertex Pharmaceuticals', 'AppLovin', 'Strategy \n (MicroStrategy)', 'Nike', 'Lockheed Martin', 'Starbucks', 'Marsh & McLennan Companies', 'Intercontinental Exchange', 'American Tower', 'DoorDash']
Ticker_US=['NVDA', 'MSFT', 'AAPL', 'AMZN', 'GOOG', 'META', 'AVGO', 'BRK-B', 'TSLA', 'JPM', 'WMT', 'V', 'LLY', 'ORCL', 'NFLX', 'MA', 'XOM', 'COST', 'PG', 'JNJ', 'HD', 'BAC', 'ABBV', 'PLTR', 'KO', 'UNH', 'PM', 'CSCO', 'TMUS', 'WFC', 'IBM', 'GE', 'CRM', 'CVX', 'ABT', 'MS', 'AXP', 'AMD', 'DIS', 'GS', 'INTU', 'NOW', 'MCD', 'T', 'MRK', 'TXN', 'UBER', 'ISRG', 'RTX', 'BX', 'CAT', 'BKNG', 'PEP', 'VZ', 'QCOM', 'BLK', 'SCHW', 'C', 'BA', 'SPGI', 'TMO', 'ADBE', 'AMGN', 'HON', 'BSX', 'PGR', 'AMAT', 'NEE', 'SYK', 'DHR', 'PFE', 'COF', 'UNP', 'GEV', 'DE', 'TJX', 'GILD', 'MU', 'PANW', 'CMCSA', 'ANET', 'KKR', 'CRWD', 'LOW', 'LRCX', 'ADP', 'KLAC', 'ADI', 'APH', 'COP', 'VRTX', 'APP', 'MSTR', 'NKE', 'LMT', 'SBUX', 'MMC', 'ICE', 'AMT', 'DASH']


한국, 미국기업 모두 같은 테이블에 저장할거다

In [ ]:
all_names = Name_KR + Name_US # 기업 명
all_tickers = Ticker_KR + Ticker_US # 티커 명

In [ ]:
company_summaries = [] # 최종 결과 저장

for name, ticker in zip(all_names, all_tickers):
    stock = yf.Ticker(ticker) # 티커 기준으로 불러오기
    summary = stock.info.get('longBusinessSummary', pd.NA) # 기업 설명 가져오기
    company_summaries.append({ #각각의 결과를 추가
        'Name': name,
        'Ticker': ticker,
        'Summary': summary
    })

In [10]:
results_df = pd.DataFrame(company_summaries)

In [11]:
results_df.head()

,Name,Ticker,Summary
0,Samsung,005930.KS,"Samsung Electronics Co., Ltd. engages in the c..."
1,SK Hynix,000660.KS,"SK hynix Inc., together with its subsidiaries,..."
2,Samsung Biologics,207940.KS,"Samsung Biologics Co.,Ltd., together with its ..."
3,LG Energy Solution,373220.KS,"LG Energy Solution, Ltd. provides energy solut..."
4,Hyundai,HYMTF,"Hyundai Motor Company, together with its subsi..."


In [12]:
results_df.isna().sum()

Name       0
Ticker     0
Summary    1
dtype: int64

# 임베딩

In [ ]:
API_KEY = os.environ.get("GEMINI_API_KEY") # Gemini API 키를 환경 변수에서 가져오기

def get_summary_embedding(summary_text: str, client: genai.Client) -> list[float] | None:
    """
    하나의 텍스트를 받아 임베딩 벡터를 반환하는 함수.
    """
    # 내용이 비어있는지 확인
    if not summary_text or pd.isna(summary_text):
        return None
    
    try:
        # API 호출
        result = client.models.embed_content(
            model="models/text-embedding-004", # 사용하는 임베딩  모델명
            contents=summary_text, # 임베딩할 내용
            config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT") # 임베딩 타입 # 검색용 문서 임베딩
        )
        # 결과 객체에서 벡터를 직접 반환
        vectors = [obj.values for obj in result.embeddings]
        vectors=vectors[0]

        return vectors
    
    except Exception as e:
        print(f"API Error embedding '{summary_text[:50]}...': {e}")
        return None


# API 실행
if __name__ == "__main__":
    # genai.Client 객체 생성
    client = genai.Client(api_key=API_KEY)
    # 결과 데이터프레임에 저장
    results_df['embedding'] = results_df['Summary'].apply(lambda text: get_summary_embedding(text, client))

In [25]:
results_df.isna().sum()

Name         0
Ticker       0
Summary      0
embedding    0
dtype: int64

# 업로드

In [26]:
results_df.rename(columns={
    'Name': 'company_name',
    'Ticker': 'ticker',
    'Summary': 'summary',
    "embedding": "summary_embedding",
}, inplace=True)
results=results_df[['company_name', 'ticker', 'summary', 'summary_embedding']].copy()

In [27]:
data_to_upload = results.to_dict('records')

supabase의 company_summary 테이블에 전체 업로드

In [28]:
data, count = supabase.table("company_summary").insert(data_to_upload).execute()